In [54]:
import random
import numpy as np
diceg = 0
dicech = 1
p = {}
p[diceg] = 0.05
p[dicech] = 0.04
pd = {}
pd[diceg] = [1/6 for i in range(6)]
pd[dicech] = [1/10 for i in range(5)] + [0.5]

In [55]:
def generate_rolls(dice):
    for i in range(10000):
        if random.random() < p[dice]:
            dice = (dice + 1) % 2
        res = np.random.choice(6, p = pd[dice]) + 1
        data.append(res)
        dices.append(dice)

In [56]:
data = []
dices = []
generate_rolls(diceg)

In [66]:
def heuristic(data, length):
    ans = []
    for i in range(len(data)):
        cluster = data[max(i-length, 0):min(i+length, len(data))]
        sixes = 0
        for c in cluster:
            if c == 6:
                sixes += 1
        if sixes > 0.32 * len(cluster):
            ans.append(dicech)
        else:
            ans.append(diceg)
    return ans  
def score(guess):
    ans = 0
    for i in range(10000):
        if guess[i] == dices[i]:
            ans+=1
    return ans / len(guess)

In [58]:
for i in range(1,20):
    h_ans = heuristic(data, i)
    print(i,":",score(h_ans))

1 : 0.7205
2 : 0.7422
3 : 0.7936
4 : 0.8053
5 : 0.802
6 : 0.8229
7 : 0.8295
8 : 0.8224
9 : 0.828
10 : 0.8222
11 : 0.8142
12 : 0.8169
13 : 0.8055
14 : 0.8066
15 : 0.7999
16 : 0.7936
17 : 0.7974
18 : 0.7925
19 : 0.7803


In [59]:
print(score(heuristic(data, 6)))

0.8229


In [60]:
p_transition = [[1- p[dicech], p[dicech]],[p[diceg], 1 - p[diceg]]] # tablica przejsc
#emisja bedzie rzut koscia z odpowiednimi prawdopodobienstwami pd 
#- upraszczamy do emitowania w danym stanie
emision = [pd[0],pd[1]]

In [61]:
def forward_backward(p_transition, emision):
    alpha = [[0 for i in range(10000)],
            [0 for i in range(10000)]] # p-stwo, ze w i-tym kroku bedziemy w (0,1) stanie
    alpha[0][0] = 1
    alpha[0][1] = 0 # bo startujemy z kostki 'poprawnej'
    def count_alpha(T):
        for t in range(1,T):
            for j in range(2):
                a0 = alpha[0][t-1] * p_transition[0][j] * emision[j][int(data[t] - 1)]
                a1 = alpha[1][t-1] * p_transition[1][j] * emision[j][int(data[t] - 1)]
                alpha[j][t] = a0 + a1  
                #alpha[j][t] *= 6
            n = alpha[0][t] + alpha[1][t]
            alpha[0][t] = alpha[0][t] / n
            alpha[1][t] = alpha[1][t] / n

    beta = [[0 for i in range(10001)],
            [0 for i in range(10001)]] # p-stwo, ze w i-tym kroku bedziemy w (0,1) stanie
    beta[0][-1] = 1
    beta[1][-1] = 1
    def count_beta(T):
        for t in range(T-1,-1,-1):
            for j in range(2):
                b0 = beta[0][t+1] * p_transition[j][0] * emision[0][int(data[t] - 1)]
                b1 = beta[1][t+1] * p_transition[j][1] * emision[1][int(data[t] - 1)]
                beta[j][t] = b0 + b1  
                #beta[j][t] *= 6
            n = beta[0][t] + beta[1][t]
            beta[0][t] = beta[0][t] / n
            beta[1][t] = beta[1][t] / n
    count_alpha(10000)
    count_beta(10000)
    ab = [[0 for i in range(10000)],
        [0 for i in range(10000)]]
    for i in range(len(ab[0])):
        n = alpha[0][i] * beta[0][i] + alpha[1][i] * beta[1][i]
        ab[0][i] = alpha[0][i] * beta[0][i] / n
        ab[1][i] = alpha[1][i] * beta[1][i] / n
    return alpha, beta, ab

In [62]:
a,b,g = forward_backward(p_transition, emision)

In [63]:
def gen_dices(m):
    ans = []
    for i in range(len(m[0])):
        if m[0][i] > m[1][i]:
            ans.append(0)
        else:
            ans.append(1)
    return ans

In [67]:
print(score(gen_dices(a)))
print(score(gen_dices(b)))
print(score(gen_dices(g)))

0.7777
0.7861213878612139
0.8321
